In [1]:
import pandas as pd

import os
import sys
import numpy as np

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

# to save the audio files
import soundfile as sf

# import keras
# from keras.callbacks import ReduceLROnPlateau
# from keras.models import Sequential
# from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from keras.utils import np_utils, to_categorical
# from keras.callbacks import ModelCheckpoint

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
# Paths for data.
Ravdess = "datasets/Ravdess/audio_speech_actors_01-24/"
Crema = "datasets/Crema/"
Tess = "datasets/Tess/"
Savee = "datasets/Savee/"

# Trimming and combining (1sec)

In [3]:
ravdess_directory_list = os.listdir(Ravdess)
crema_directory_list = os.listdir(Crema)
tess_directory_list = os.listdir(Tess)
savee_directory_list = os.listdir(Savee)

segment_length = 1
sampling_rate = 22050

## Ravdess

In [5]:
actor_counter = 0

# Loop through each file and get the length
for dir in ravdess_directory_list:
    actor = os.listdir(Ravdess + dir)
    actor_samples_counter = 0

    for file in actor:

        # dir = ravdess_directory_list[0]
        # file = os.listdir(Ravdess + dir)[0]

        part = file.split('.')[0]
        part = part.split('-')
        if int(part[2]) == 1 or int(part[2]) == 2:
            path = "datasets/combined_L/neutral+calm/"
        elif int(part[2]) == 3:
            path = "datasets/combined_L/happy/"
        elif int(part[2]) == 4:
            path = "datasets/combined_L/sad/"
        elif int(part[2]) == 5:
            path = "datasets/combined_L/angry/"
        elif int(part[2]) == 6:
            path = "datasets/combined_L/fear/"
        elif int(part[2]) == 7:
            path = "datasets/combined_L/disgust/"
        elif int(part[2]) == 8:
            path = "datasets/combined_L/surprised/"
        else:
            path = "datasets/combined_L/"

        y, sr = librosa.load(Ravdess + dir + '/' + file)
        y_trimmed, _ = librosa.effects.trim(y, top_db=50)
        normalized_audio = librosa.util.normalize(y_trimmed)

        segment_length_samples = int(segment_length * sr)
        num_segments = -(-len(normalized_audio) // segment_length_samples)

        for i in range(num_segments):
            start_sample = i * segment_length_samples
            end_sample = start_sample + segment_length_samples
            segment = normalized_audio[start_sample:end_sample]
            
            seg_ln = librosa.get_duration(y=segment, sr=sr)
            if seg_ln > 0.2:
                if segment_length > seg_ln:
                    segment = librosa.util.fix_length(segment, size=sampling_rate)
            
                output_file_path = os.path.join(path, f'ravdess_{actor_counter}_{actor_samples_counter}_{i}.wav')
                sf.write(output_file_path, segment, sr)

        actor_samples_counter += 1
    actor_counter += 1


## Crema

In [6]:
file_counter = 0

for file in crema_directory_list:
    part = file.split('_')
    if part[2] == 'SAD':
        path = "datasets/combined_L/sad/"
    elif part[2] == 'ANG':
        path = "datasets/combined_L/angry/"
    elif part[2] == 'DIS':
        path = "datasets/combined_L/disgust/"
    elif part[2] == 'FEA':
        path = "datasets/combined_L/fear/"
    elif part[2] == 'HAP':
        path = "datasets/combined_L/happy/"
    elif part[2] == 'NEU':
        path = "datasets/combined_L/neutral+calm/"
    else:
        path = "datasets/combined_L/"

    y, sr = librosa.load(Crema + file)
    y_trimmed, _ = librosa.effects.trim(y, top_db=25)
    normalized_audio = librosa.util.normalize(y_trimmed)

    # Segment length in samples
    segment_length_samples = int(segment_length * sr)

    # Number of segments
    num_segments = -(-len(normalized_audio) // segment_length_samples)

    for i in range(num_segments):
        start_sample = i * segment_length_samples
        end_sample = start_sample + segment_length_samples
        segment = normalized_audio[start_sample:end_sample]
        
        seg_ln = librosa.get_duration(y=segment, sr=sr)
        if seg_ln > 0.2:
            if segment_length > seg_ln:
                segment = librosa.util.fix_length(segment, size=sampling_rate)
        
            output_file_path = os.path.join(path, f'crema_{file_counter}_{i}.wav')
            sf.write(output_file_path, segment, sr)

    file_counter += 1


## Tess

In [7]:
# tess
folder_counter = 0

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    file_counter = 0
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]

        if part == "angry":
            path = "datasets/combined_L/angry/"
        elif part == "disgust":
            path = "datasets/combined_L/disgust/"
        elif part == "fear":
            path = "datasets/combined_L/fear/"
        elif part == "happy":
            path = "datasets/combined_L/happy/"
        elif part == "neutral":
            path = "datasets/combined_L/neutral+calm/"
        elif part == 'ps':
            path = "datasets/combined_L/surprised/"
        elif part == 'sad':
            path = "datasets/combined_L/sad/"
        else:
            path = "datasets/combined_L/"

        y, sr = librosa.load(Tess + dir + '/' + file)
        y_trimmed, _ = librosa.effects.trim(y)
        normalized_audio = librosa.util.normalize(y_trimmed)

        # Segment length in samples
        segment_length_samples = int(segment_length * sr)

        # Number of segments
        num_segments = -(-len(normalized_audio) // segment_length_samples)

        for i in range(num_segments):
            start_sample = i * segment_length_samples
            end_sample = start_sample + segment_length_samples
            segment = normalized_audio[start_sample:end_sample]
            
            seg_ln = librosa.get_duration(y=segment, sr=sr)
            if seg_ln > 0.2:
                if segment_length > seg_ln:
                    segment = librosa.util.fix_length(segment, size=sampling_rate)
            
                output_file_path = os.path.join(path, f'tess_{folder_counter}_{file_counter}_{i}.wav')
                sf.write(output_file_path, segment, sr)

        file_counter += 1
    folder_counter += 1

## Savee

In [8]:
# savee
file_counter = 0

for file in savee_directory_list:
    part = file.split('_')[1]
    ele = part[:-6]
    if ele == 'a':
        path = "datasets/combined_L/angry/"
    elif ele == 'd':
        path = "datasets/combined_L/disgust/"
    elif ele == 'f':
        path = "datasets/combined_L/fear/"
    elif ele == 'h':
        path = "datasets/combined_L/happy/"
    elif ele == 'n':
        path = "datasets/combined_L/neutral+calm/"
    elif ele == 'sa':
        path = "datasets/combined_L/sad/"
    elif ele == 'su':
        path = "datasets/combined_L/surprised/"
    else:
        path = "datasets/combined_L/"

    y, sr = librosa.load(Savee + file)
    y_trimmed, _ = librosa.effects.trim(y, top_db=20)
    normalized_audio = librosa.util.normalize(y_trimmed)

    # Segment length in samples
    segment_length_samples = int(segment_length * sr)

    # Number of segments
    num_segments = -(-len(normalized_audio) // segment_length_samples)

    for i in range(num_segments):
        start_sample = i * segment_length_samples
        end_sample = start_sample + segment_length_samples
        segment = normalized_audio[start_sample:end_sample]
        
        seg_ln = librosa.get_duration(y=segment, sr=sr)
        if seg_ln > 0.2:
            if segment_length > seg_ln:
                segment = librosa.util.fix_length(segment, size=sampling_rate)
        
            output_file_path = os.path.join(path, f'savee_{file_counter}_{i}.wav')
            sf.write(output_file_path, segment, sr)

    file_counter += 1


# Data Augmentation

In [10]:
import librosa
import numpy as np
import random

# Individual augmentation functions
def pitch_shift(audio, sampling_rate, n_steps):
    return librosa.effects.pitch_shift(audio, sr=sampling_rate, n_steps=n_steps)

def add_noise(audio, noise_factor):
    noise_amp = noise_factor*np.random.uniform()*np.amax(audio)
    audio = audio + noise_amp*np.random.normal(size=audio.shape[0])
    return audio

def frequency_masking(audio, sr, freq_mask_param=15):
    spectrogram = librosa.stft(audio)
    spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram))
    
    freq_mask = np.random.randint(0, spectrogram_db.shape[0] - freq_mask_param)
    spectrogram_db[freq_mask:freq_mask + freq_mask_param, :] = 0
    
    masked_audio = librosa.istft(librosa.db_to_amplitude(spectrogram_db))
    return masked_audio

def time_masking(audio, sr, time_mask_param=30):
    spectrogram = librosa.stft(audio)
    spectrogram_db = librosa.amplitude_to_db(np.abs(spectrogram))
    
    time_mask = np.random.randint(0, spectrogram_db.shape[1] - time_mask_param)
    spectrogram_db[:, time_mask:time_mask + time_mask_param] = 0
    
    masked_audio = librosa.istft(librosa.db_to_amplitude(spectrogram_db))
    return masked_audio

# Combination augmentation function
def combine_augmentations(audio, sr):
    if random.random() < 0.5:
        audio = pitch_shift(audio, sr, random.randint(-2, 2))
    if random.random() < 0.5:
        audio = add_noise(audio, 0.035)
    if random.random() < 0.5:
        audio = frequency_masking(audio, sr)
    if random.random() < 0.5:
        audio = time_masking(audio, sr)
    return audio


In [11]:
path = 'datasets/combined_L/surprised'
directory_list = os.listdir(path)
# file = directory_list[0]

for file in directory_list:
    y, sr = librosa.load(path + "/" + file)
    
    # Apply individual augmentations
    pitch_shifted_audio = pitch_shift(y, sr, 1)
    noisy_audio = add_noise(y, 0.035)
    # freq_masked_audio = frequency_masking(y, sr)
    # time_masked_audio = time_masking(y, sr)
    
    # Apply combination augmentations
    # combined_augmented_audio = combine_augmentations(y, sr)
    
    sf.write(os.path.join(path, f'shifted_{file}'), pitch_shifted_audio, sr)
    sf.write(os.path.join(path, f'noisy_{file}'), noisy_audio, sr)
    
    # pd.Series(pitch_shifted_audio).plot(figsize=(10, 5), lw=1, title='Raw audio data Tess (normalized)')
    # librosa.get_duration(y=pitch_shifted_audio, sr=sr)
    # Audio(pitch_shifted_audio, rate=sr)

## Feature extraction

In [13]:
import librosa.feature

def extract_zcr(audio):
    zcr = librosa.feature.zero_crossing_rate(y=audio)
    zcr_mean = np.mean(zcr.T, axis=0)
    # print("extract_zcr", zcr.shape, zcr_mean.shape)
    return zcr_mean

def extract_chroma(audio, sr):
    chroma = librosa.feature.chroma_stft(S=audio, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)
    # print("extract_chroma", chroma.shape, chroma_mean.shape)
    return chroma_mean

def extract_mfccs(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    # print("extract_mfccs", mfccs.shape, mfccs_mean.shape)
    return mfccs_mean

def extract_spectral_contrast(audio, sr):
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast.T, axis=0)
    # print("extract_spectral_contrast", spectral_contrast.shape, spectral_contrast_mean.shape)
    return spectral_contrast_mean

def extract_spectral_rolloff(audio, sr):
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
    spectral_rolloff_mean = np.mean(spectral_rolloff.T, axis=0)
    # print("extract_spectral_rolloff", spectral_rolloff.shape, spectral_rolloff_mean.shape)
    return spectral_rolloff_mean

def extract_rmse(audio):
    rmse = librosa.feature.rms(y=audio)
    rmse_mean = np.mean(rmse.T, axis=0)
    # print("extract_rmse", rmse.shape, rmse_mean.shape)
    return rmse_mean

def extract_mel_spectrogram(audio, sr):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    mel_spectrogram_mean = np.mean(mel_spectrogram.T, axis=0)
    # print("extract_mel_spectrogram", mel_spectrogram.shape, mel_spectrogram_mean.shape)
    return mel_spectrogram_mean

def extract_features(data, sample_rate):
    result = np.array([])
    
    # ZCR
    zcr = extract_zcr(data)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = extract_chroma(stft, sample_rate)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = extract_mfccs(data, sample_rate)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Spectral_contrast
    spectral_contrast = extract_spectral_contrast(data, sample_rate)
    result = np.hstack((result, spectral_contrast)) # stacking horizontally

    # Spectral_rolloff
    spectral_rolloff = extract_spectral_rolloff(data, sample_rate)
    result = np.hstack((result, spectral_rolloff)) # stacking horizontally

    # RMS
    rms = extract_rmse(data)
    result = np.hstack((result, rms)) # stacking horizontally

    # Mel_spectrogram
    mel_spectrogram = extract_mel_spectrogram(data, sample_rate)
    result = np.hstack((result, mel_spectrogram)) # stacking horizontally
    
    return result

def get_features(path):
    data, sample_rate = librosa.load(path)
    
    features = extract_features(data, sample_rate)
    result = np.array(features)
    
    return result


In [14]:
features = get_features("datasets/combined_L/angry/crema_0_0.wav")
features.shape

(170,)

In [15]:
import os
import numpy as np
import pandas as pd

path = "datasets/combined_L"
directory_list = os.listdir(path)
X = []
angry_labels = []
disgust_labels = []
fear_labels = []
happy_labels = []
neutral_labels = []
sad_labels = []
surprised_labels = []

for emotion in directory_list:
    angry_label = 0
    disgust_label = 0
    fear_label = 0
    happy_label = 0
    neutral_label = 0
    sad_label = 0
    surprised_label = 0
    match emotion:
        case "angry":
            angry_label = 1
        case "disgust":
            disgust_label = 1
        case "fear":
            fear_label = 1
        case "happy":
            happy_label = 1
        case "neutral+calm":
            neutral_label = 1
        case "sad":
            sad_label = 1
        case "surprised":
            surprised_label = 1
        
    samples = os.listdir(path + "/" + emotion)
    for sample in samples:
        features = get_features(path + "/" + emotion + "/" + sample)
        X.append(features)
        angry_labels.append(angry_label)
        disgust_labels.append(disgust_label)
        fear_labels.append(fear_label)
        happy_labels.append(happy_label)
        neutral_labels.append(neutral_label)
        sad_labels.append(sad_label)
        surprised_labels.append(surprised_label)
    
Features = pd.DataFrame(X)
Features['angry'] = angry_labels
Features['disgust'] = disgust_labels
Features['fear'] = fear_labels
Features['happy'] = happy_labels
Features['neutral'] = neutral_labels
Features['sad'] = sad_labels
Features['surprised'] = surprised_labels
Features.to_csv('datasets/dataset_L.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,167,168,169,angry,disgust,fear,happy,neutral,sad,surprised
0,0.105047,0.575523,0.570168,0.629563,0.657737,0.675737,0.471088,0.477760,0.565239,0.545473,...,1.381563e-07,1.360737e-07,1.348835e-07,1,0,0,0,0,0,0
1,0.161821,0.368584,0.428432,0.539139,0.590300,0.394862,0.391811,0.499014,0.623780,0.721728,...,6.980856e-06,6.932686e-06,6.897007e-06,1,0,0,0,0,0,0
2,0.048495,0.416398,0.456305,0.366213,0.373536,0.409270,0.357111,0.363098,0.395970,0.506166,...,7.347799e-06,7.279295e-06,7.233022e-06,1,0,0,0,0,0,0
3,0.086381,0.365141,0.354305,0.494114,0.693808,0.680345,0.539391,0.460055,0.496273,0.441196,...,1.869142e-06,1.863559e-06,1.861637e-06,1,0,0,0,0,0,0
4,0.074174,0.314639,0.372686,0.397095,0.391753,0.489682,0.519263,0.524107,0.475452,0.485388,...,1.027138e-05,1.013511e-05,1.005470e-05,1,0,0,0,0,0,0


In [38]:
Features.shape

(43830, 171)